In [1]:
#import necessary libraries 
import pandas as pd
import numpy as np
import twython
import itertools
from itertools import combinations
from twython import Twython
import string

In [2]:
#Fetching tweets using Twyton for particular hash tags id given in the data set attached 
# need to create your own twitter id for fectching the tweets using the beloww
twitter = Twython(

    'vv3A6qBhIggsBB9mQW7WiFC7M', 'mxwxmMuTexwORxPT3YWJtx0mDuSFtZy6YsYJdUIyjireFumjFY',

    '2894451055-7IG33DjAAKswedvumGyCNMxlPVh1XvuzE8BlvOP', 'UJt2PLY2opW9XxyUHB55mGfINahLL5M1dbpXNhVma3DfT')

In [12]:
# import the csv file containg Id and hash Tags and craeate a dummy data frame as newDF
df = pd.read_csv('D://Projects//Gunjan//dataset//dataset/train_data.csv', names=['ID', 'Hash_Tags'],dtype={'Hash_Tags' : 'category'})
newDF = pd.DataFrame(columns=['ID','Hash_Tags','Tweets'])

In [13]:
df.head()

,ID,Hash_Tags
0,564818124352917505,#shamitabh
1,728820161789214720,NIL
2,722811421663367168,NIL
3,565061873448390656,#aapsweep
4,567360176127107072,NIL


In [14]:
# for subsetting based on rows say 1000 top rows as u can handle not more than 100o tweets approximately at a time
df = df.ix[1:1000]
#df.tail()

In [15]:
#For combing the fetched tweets with dataframe df containg Id And Hashtags 
for index, row in df.iterrows():

    try:

        tweet = twitter.show_status(id=row['ID'])

        newDF.set_value(index,'ID',row['ID'])

        newDF.set_value(index,'Hash_Tags',row['Hash_Tags'])

        newDF.set_value(index,'Tweets',tweet['text'])

    except Exception:

        pass

    


In [16]:
# the new dataframe newDF 
newDF.head()

,ID,Hash_Tags,Tweets
1,728820161789214720,NIL,Karma is a bitch😎
2,722811421663367168,NIL,RT @NigamAmbika: @bhardwaj_kamini @_KaranMehra...
3,565061873448390656,#aapsweep,RT @rajatub: Resounding AAP victory reminds me...
4,567360176127107072,NIL,RT @ohmnamahshiva: @rmburra24 ohm namah shivay...
5,707550604516691968,#jnu,RT @ABVPVoice: Akhil Bhartiya Vidyarthi Parish...


In [17]:
#newDF.Hash_Tags.unique()
newDF.shape

(863, 3)

In [18]:
#finding the unique hash tags category
newDF['Hash_Tags'].value_counts()

NIL                353
#indvssa           251
#railbudget2015     84
#netneutrality      72
#aapsweep           43
#blockbustermsg     21
#shamitabh          18
#jnu                12
#rahulonleave        6
#mahashivratri       3
Name: Hash_Tags, dtype: int64

In [19]:
#removing rows with null values
newDFF = newDF[newDF.Hash_Tags != 'NIL']

In [20]:
newDFF.isnull().sum()

ID           0
Hash_Tags    0
Tweets       0
dtype: int64

In [21]:
import nltk
#nltk.download('all')
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [3]:
# # removing stopwords ,converting to lowercase ,removing punctuation and other necessary pre processing 
# newDFF['Tweets'] = newDFF['Tweets'].str.lower()
# newDFF['cleaned'] = newDFF['Tweets'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
# newDFF['cleaned'] = newDFF['cleaned'].str.replace('\d+', '')
# newDFF['cleaned'] = newDFF['cleaned'].map(lambda x: x.strip(''))
# newDFF['cleaned'].apply(lambda x: [item for item in x if item not in stop])

In [59]:
print(newDFF.Tweets[1:10])
print(newDFF.cleaned[1:10])

5     rt @abvpvoice: akhil bhartiya vidyarthi parish...
8                       run-out chance miss :( #indvssa
9     hey @marcorubio @repedwhitfield, richard r. of...
12    hey @senronjohnson @michaelcburgess, d'arcy g....
13    explaining how cricket works to a german. omg!...
17    namah parvati pate har har mahadev #mahashivratri
21    guruji has done a wonderful job to eradicate s...
24    hey @peteolson @repjoebarton, paul k. of wv se...
27    @hannelislabber we are indians 😎 \nthe show i...
Name: Tweets, dtype: object
5     rt abvpvoice akhil bhartiya vidyarthi parishad...
8                           runout chance miss  indvssa
9     hey marcorubio repedwhitfield richard r of me ...
12    hey senronjohnson michaelcburgess darcy g of n...
13    explaining how cricket works to a german omg\n...
17     namah parvati pate har har mahadev mahashivratri
21    guruji has done a wonderful job to eradicate s...
24    hey peteolson repjoebarton paul k of wv sez fr...
27    hannelislabber 

In [39]:
#  creating teh term document frequency matrix
from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer()
countvec.fit_transform(newDFF['cleaned'])
pd.DataFrame(countvec.fit_transform(newDFF['cleaned']).toarray(), columns=countvec.get_feature_names())


<678x2652 sparse matrix of type '<type 'numpy.int64'>'
	with 8670 stored elements in Compressed Sparse Row format>

In [43]:
# doing the necessary hot encoding as is necessary for using ML algorithms in python
from sklearn import preprocessing
# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()
# Convert marital variable to numeric
encoded_tags = label_encoder.fit_transform(newDFF['Hash_Tags'])

final = pd.DataFrame([newDFF['ID'], encoded_tags, newDFF['Tweets']]).T
final.head()



,ID,Unnamed 0,Tweets
3,565061873448390656,5,rt @rajatub: resounding aap victory reminds me...
5,707550604516691968,2,rt @abvpvoice: akhil bhartiya vidyarthi parish...
8,569415946906898432,5,run-out chance miss :( #indvssa
9,706757802421788673,2,"hey @marcorubio @repedwhitfield, richard r. of..."
12,709422593418403840,5,"hey @senronjohnson @michaelcburgess, d'arcy g...."


In [ ]:
# dpoing the encoding manually to know which category is assigned what values
newDFF['Hash_Tags'].replace(0, '#indvssa ',inplace=True)
newDFF['Hash_Tags'].replace(1, '#railbudget2015 ',inplace=True)
newDFF['Hash_Tags'].replace(2, '#netneutrality  ',inplace=True)
newDFF['Hash_Tags'].replace(3, '#aapsweep',inplace=True)
newDFF['Hash_Tags'].replace(4, '#blockbustermsg',inplace=True)
newDFF['Hash_Tags'].replace(5, '#shamitabh',inplace=True)
newDFF['Hash_Tags'].replace(6, '#jnu ',inplace=True)
newDFF['Hash_Tags'].replace(7, '#rahulonleave',inplace=True)
newDFF['Hash_Tags'].replace(8, '#mahashivratri  ',inplace=True)

 

In [ ]:
# svm Model Building 
model = svm.svc(kernel='linear', c=1, gamma=1)

In [ ]:
# here X will newDFF without the hastags column ,y is the hastag column of newDFF,X_test is the test data
model.fit(X, y)
model.score(X, y)
predicted= model.predict(x_test)